In [2]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from pathlib import Path
import faiss

c:\Users\Kistanov\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
chunk_files = sorted(Path('.').glob('preproc_data*.parquet'))
dfs = [pd.read_parquet(f) for f in chunk_files]
df = pd.concat(dfs, ignore_index=True)

In [6]:
texts = df['lemmatized_text'].tolist() 


In [7]:
model = SentenceTransformer('cointegrated/rubert-tiny2')

embeddings = model.encode(texts, show_progress_bar=True, batch_size=64) # create embeddings
embeddings = np.array(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True) # normalize it


c:\Users\Kistanov\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kistanov\.cache\huggingface\hub\models--cointegrated--rubert-tiny2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Batches: 100%|██████████| 777/777 [47:14<00:00,  3.65s/it]  


In [13]:
index = faiss.IndexFlatIP(embeddings.shape[1])  # create index
index.add(embeddings)  # add embeddings to index
faiss.write_index(index,"embeddings.index")  # save index to file

In [14]:
emb = faiss.read_index("embeddings.index")  # read index from file

In [15]:
emb

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000001E7DE4738D0> >